In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import options_trading as opt
import pricing
%load_ext autoreload
%autoreload 2

### Simulate Asset Price with Monte Carlo

In [4]:
S0=100
K=100
T=1
H=125
r=0.01
vol=0.2

N=100   #number of timesteps
M=1000  #number of simulations


def monte_carlo(S0, T, r, vol, N, M):
    dt=T/N
    nudt=(r-0.5*vol**2)*dt
    volsdt=vol*np.sqrt(dt)
    erdt=np.exp(-r*dt)

    S=pd.DataFrame()

    for i in range(M):
        St=pd.Series(index=np.arange(0,N,1)).rename(f"{i+1}")
        St[0]=S0
    
        for j in range (1, N+1):
            epsilon=np.random.normal()
            Stn=St[j-1]*np.exp(nudt+volsdt*epsilon)
            St[j]=Stn

        S=pd.concat([S, St], axis=1)

    return S

In [21]:
monte_carlo(S0, T, r, vol, N, M)

,1,2,3,4,5,6,7,8,9,10,...,991,992,993,994,995,996,997,998,999,1000
0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
1,97.768375,99.601480,101.200619,98.111650,100.566282,99.329402,99.720696,98.714177,97.698759,100.685197,...,99.531524,98.343637,99.855900,99.544698,101.441600,99.591017,98.190810,95.944904,96.698467,97.856663
2,96.662416,98.745168,99.847459,100.880969,100.026409,101.458658,97.572408,96.996126,97.377983,104.309104,...,100.195219,98.317873,96.260970,97.416198,104.991198,100.533393,96.796618,95.525060,101.217434,99.163967
3,97.075415,98.130584,97.206581,99.873091,99.201466,103.033034,96.414278,99.801754,100.829797,106.663421,...,100.490432,98.436115,97.773705,95.976554,103.661108,97.849241,95.080717,92.626285,102.424782,99.060176
4,93.974248,100.326699,98.995034,100.915525,99.477025,103.020489,101.508326,97.403046,97.294758,109.923230,...,102.391508,95.274630,99.776252,97.224124,106.218132,100.249638,99.173547,95.322871,102.590013,99.857230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,102.124115,72.186882,115.324392,105.489562,106.303751,88.178460,78.833397,94.348529,76.817163,87.738056,...,129.259954,82.021297,113.011152,84.512888,73.440301,62.425646,85.040501,73.452742,98.652934,93.165117
97,104.250632,72.268594,116.673187,106.407286,105.449308,85.464734,80.061708,95.017023,78.405511,86.540891,...,133.793208,81.893359,110.624936,81.533980,71.753879,60.154770,85.744623,71.675640,97.271192,92.099071
98,107.043062,71.750945,116.413698,112.480605,104.428249,86.084760,81.233864,93.464098,75.625295,88.295074,...,134.819533,83.254245,109.864022,81.080742,73.055211,60.736319,82.693894,72.148795,98.522167,90.492620
99,105.664800,72.040714,119.222751,110.770700,102.912362,85.327074,80.725419,89.481015,73.847030,90.533005,...,134.993776,84.660843,108.150486,83.619912,74.648439,60.469882,81.442846,71.632702,100.740550,92.208444


In [5]:
S0=100
K=100
T=1
H=125
r=0.01
vol=0.2

N=100   #number of timesteps
M=1000  #number of simulations


def monte_carlo_vectorized(S0, T, r, vol, N, M):
    dt=T/N
    nudt=(r-0.5*vol**2)*dt
    volsdt=vol*np.sqrt(dt)
    erdt=np.exp(-r*dt)
    epsilon=np.random.normal(0, 1, (N, M))
    growth_factors=np.exp(nudt + volsdt * epsilon)
    prices=S0*pd.DataFrame(growth_factors).cumprod(axis=0)
    S=pd.concat([pd.DataFrame([S0] * M).T, prices], ignore_index=True)
    S.columns=[f"{i+1}" for i in range(M)]

    return S

In [41]:
monte_carlo_vectorized(S0, T, r, vol, N, M)

,1,2,3,4,5,6,7,8,9,10,...,991,992,993,994,995,996,997,998,999,1000
0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
1,103.173628,100.463804,99.658814,99.049794,99.179722,103.262355,99.215397,101.674587,100.931740,99.111912,...,102.135714,96.408091,100.610130,95.496188,102.072997,98.274853,97.371966,100.240349,97.781324,95.322261
2,103.420467,100.051503,99.400243,101.938384,96.208126,104.678541,99.001388,102.167207,101.703398,99.197814,...,101.712879,98.246926,99.677793,95.434493,102.831800,96.453932,97.766927,98.327061,98.335649,96.130126
3,103.960809,98.638400,101.781947,101.489946,97.252158,104.832515,101.247727,104.299167,103.373264,100.570755,...,103.291594,98.612306,99.836038,96.431487,101.474881,93.422061,100.336589,99.110184,101.880131,95.351038
4,107.174775,99.321258,104.207049,99.963309,95.433139,104.266843,98.679159,105.874454,102.475095,101.375663,...,102.603978,102.108315,98.966104,91.845979,100.044456,90.028491,98.308208,97.915551,104.818464,93.963429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,105.748701,121.183952,133.621502,95.621142,126.402783,106.076670,77.379327,101.173347,96.311585,101.666194,...,154.027653,114.271977,88.952614,107.953102,79.654614,69.443726,81.386784,99.485901,102.625030,80.151739
97,106.596490,122.736742,130.743643,93.604200,125.180221,106.065461,76.247234,101.660788,97.896887,102.076820,...,151.183022,115.152221,90.670447,108.469553,80.673160,68.889499,79.851842,101.728611,106.316412,81.246164
98,107.896317,122.399842,126.455007,95.687896,126.884982,106.516258,74.164900,98.079058,94.440636,102.922204,...,145.631997,119.843523,91.497408,104.343868,77.897923,70.271051,79.863611,100.004216,110.128542,80.639357
99,106.798103,122.351997,128.332114,92.001281,125.725706,104.866584,71.960826,96.111337,94.798235,103.791799,...,144.236275,125.985510,89.117070,105.465817,78.264671,69.705559,80.359856,100.392560,110.795269,81.103595


### Price Vanilla Option with Monte Carlo

In [6]:
S0=100
K=100
T=1
H=125
r=0.01
vol=0.2

N=100   #number of timesteps
M=1000  #number of simulations


def monte_carlo_price(S0, K, T, r, vol, N, M):
    dt=T/N
    nudt=(r-0.5*vol**2)*dt
    volsdt=vol*np.sqrt(dt)
    erdt=np.exp(-r*dt)

    sum_CT=0
    sum_CT2=0

    for i in range(M):
        St=S0
    
        for j in range (N):
            epsilon=np.random.normal()
            Stn=St*np.exp(nudt+volsdt*epsilon)
            St=Stn
        
        CT=max(0,St-K)
            
        sum_CT=sum_CT+CT
        sum_CT2=sum_CT2+CT*CT
    
    C0=np.exp(-r*T)*sum_CT/M
    sigma=np.sqrt((sum_CT2-sum_CT*sum_CT/M)*np.exp(-2*r*T)/(M-1))
    SE=sigma/np.sqrt(M)    #Compute the standard error
    
    return (C0, SE)

In [7]:
def monte_carlo_price_vectorized(S0, K, T, r, vol, N, M, opt_type='C'):
    """
    opt_type : "C" for a call and "P" for a put
    """
    asset_prices=monte_carlo_vectorized(S0, T, r, vol, N, M)
    final_prices=asset_prices.iloc[-1, :]
    
    if opt_type=='C':
        payoffs=np.maximum(final_prices - K, 0)
    elif opt_type=='P':
        payoffs=np.maximum(K-final_prices, 0)
    else:
        print("opt_type should be either 'C' for a call option or 'P' for a put option")

    option_price=np.exp(-r*T)*payoffs.mean()

    return option_price

In [8]:
monte_carlo_price_vectorized(S0=100, K=110, T=1, r=0.02, vol=0.2, N=100, M=100000, opt_type='C')

4.920497634962092

In [9]:
opt.black_scholes(r=0.02, S=100, K=110, T=1, sigma=0.2, type="c")

4.94386695723049

### Price a digit option

In [10]:
def digit_option_price(S0, barrier, T, r, vol, digit, N, M):
    asset_prices=monte_carlo_vectorized(S0, T, r, vol, N, M)
    final_prices=asset_prices.iloc[-1, :]

    payoffs=np.where(final_prices>barrier, digit, 0)

    digit_price=np.exp(-r*T)*payoffs.mean()

    return digit_price

In [11]:
digit_option_price(S0=100, barrier=100, T=1, r=0.02, vol=0.2, digit=10, N=100, M=100000)

4.902659704278397

### Price a Barrier Option

In [ ]:
def barrier_option_price(S0, barrier, K, T, r, vol, N, M, opt_type="C", barrier_type="knock_in"):
    """
    opt_type : "C" for Call and "P" for Put
    barrier_type : "knock_in" or "knock_out"
    """
    asset_prices=monte_carlo_vectorized(S0, T, r, vol, N, M)
    final_prices=asset_prices.iloc[-1, :]

    trigger=np.where(asset_prices>barrier, 1, 0)
    trigger = np.maximum.accumulate(trigger, axis=0)

    if barrier_type=="knock_in":
        if opt_type=="C":
            payoffs=trigger[-1, :]*np.maximum(final_prices-K, 0)
        elif opt_type=="P":
            payoffs=trigger[-1, :]*np.maximum(K-final_prices, 0)
        else:
            print("opt_type shloud be either 'C' for a call or 'P' for a put")
    elif barrier_type=="knock_out":
        if opt_type=="C":
            payoffs=(1-trigger[-1, :])*np.maximum(final_prices-K, 0)
        elif opt_type=="P":
            payoffs=(1-trigger[-1, :])*np.maximum(K-final_prices, 0)
        else:
            print("opt_type shloud be either 'C' for a call or 'P' for a put")
    else:
        print("barrier_type should be either 'knock_in' or 'knock_out'")

    option_price=np.exp(-r*T)*payoffs.mean()

    return option_price

In [24]:
barrier_option_price(S0=100, barrier=110, K=115, T=1, r=0.02, vol=0.2, N=100, M=100000)

3.583705193408904

In [29]:
def barrier_option_price_v2(S0, barrier, K, T, r, vol, N, M, opt_type="C", barrier_type="knock_in"):
    """
    opt_type : "C" for Call and "P" for Put
    barrier_type : "knock_in" or "knock_out"
    """
    asset_prices = monte_carlo_vectorized(S0, T, r, vol, N, M)
    final_prices = asset_prices.iloc[-1, :]

    # Return a Series a Boolean : True if barrier was crossed and False if not
    crossed_barrier = np.any(asset_prices>barrier, axis=0)

    # Compute the right crossed_barrier Series according to the barrier_type
    if barrier_type == "knock_in":
        active_payoff = crossed_barrier
    elif barrier_type == "knock_out":
        active_payoff = ~crossed_barrier
    else:
        raise ValueError("barrier_type should be either 'knock_in' or 'knock_out'")

    # Compute the payoff depending on if it is a Call or a Put
    if opt_type == "C":
        payoffs = active_payoff * np.maximum(final_prices - K, 0)
    elif opt_type == "P":
        payoffs = active_payoff * np.maximum(K - final_prices, 0)
    else:
        raise ValueError("opt_type should be either 'C' for a call or 'P' for a put")

    # Compute the price actualized
    option_price = np.exp(-r * T) * payoffs.mean()

    return option_price

In [30]:
barrier_option_price_v2(S0=100, barrier=110, K=115, T=1, r=0.02, vol=0.2, N=100, M=100000)

3.5855722161561383